In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
import lightning as L

from lightning.pytorch.callbacks import ModelCheckpoint

from src.model.modeling_bindk import LitBINDK
from src.data.dataset import get_train_dataloader, get_dev_dataloader, get_test_dataloader
from src.metrics.metric import calculate_metric


In [3]:
SEED=42
DATASET_NAME = 'jwengr/DACON-Korean-Review-Obfuscation'
MINI_BATCH_SIZE=2
N_BATCH = 8
BASE_MODEL_NAME='Qwen/Qwen3-0.6B-Base'
EPOCHS=10
LEARNING_RATE = 5e-5
TASK='jamo'
USE_BNTD=True
TRAIN_MAX_LENGTH=128
VALID_MAX_LENGTH=128
INFERENCE_SENTENCE_MAX_LENGTH=64
INFERENCE_SENTENCE_MIN_LENGTH=32
INFERENCE_SENTENCE_N_OVERLAP=3

In [4]:
L.seed_everything(SEED)

Seed set to 42


42

In [5]:
train_dl = get_train_dataloader(DATASET_NAME, batch_size=MINI_BATCH_SIZE, max_length=TRAIN_MAX_LENGTH)
dev_dl = get_dev_dataloader(DATASET_NAME, batch_size=MINI_BATCH_SIZE, max_length=VALID_MAX_LENGTH)
test_dl = get_test_dataloader(DATASET_NAME, batch_size=1)

In [6]:
lit_bind = LitBINDK(
    base_model_name=BASE_MODEL_NAME,
    lr=LEARNING_RATE,
    epochs=EPOCHS,
    task=TASK,
    use_bntd=USE_BNTD,
    inference_sentence_max_length=INFERENCE_SENTENCE_MAX_LENGTH,
    inference_sentence_min_length=INFERENCE_SENTENCE_MIN_LENGTH,
    inference_sentence_n_overlap=INFERENCE_SENTENCE_N_OVERLAP
)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/bindk',
    filename=f"{DATASET_NAME.split('/')[1]}/{BASE_MODEL_NAME.split('/')[1]}"+"-{epoch:02d}-{valid_loss:.4f}",
    every_n_epochs=1,
    save_top_k=-1,
)

In [8]:
trainer = L.Trainer(
    callbacks=[checkpoint_callback],
    precision='16-mixed',
    max_epochs=EPOCHS,
    enable_checkpointing=True,
    accumulate_grad_batches=N_BATCH
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(lit_bind, train_dl, dev_dl)

You are using a CUDA device ('NVIDIA GeForce RTX 5070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params | Mode 
----------------------------------------
0 | bindk | BINDK | 596 M  | train
----------------------------------------
596 M     Trainable params
33.6 K    Non-trainable params
596 M     Total params
2,384.334 Total estimated model params size (MB)
1         Modules in train mode
427       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\jinwo\.virtualenvs\KROP-L3im0CPD\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
c:\Users\jinwo\.virtualenvs\KROP-L3im0CPD\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
lit_bind = LitBINDK.load_from_checkpoint(
    'checkpoints/bindk/DACON-Korean-Review-Obfuscation/Qwen3-0.6B-Base-epoch=09-valid_loss=0.0413.ckpt',
    base_model_name=BASE_MODEL_NAME,
    lr=LEARNING_RATE,
    epochs=EPOCHS,
    task=TASK,
    use_bntd=USE_BNTD,
    inference_sentence_max_length=INFERENCE_SENTENCE_MAX_LENGTH,
    inference_sentence_min_length=INFERENCE_SENTENCE_MIN_LENGTH,
    inference_sentence_n_overlap=INFERENCE_SENTENCE_N_OVERLAP
)

In [12]:
trainer = L.Trainer()

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [13]:
preds = trainer.predict(lit_bind, test_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\jinwo\.virtualenvs\KROP-L3im0CPD\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [42]:
src_sentences = [batch['sentence_noisy'][0] for batch in test_dl]
tgt_sentences = [batch['sentence'][0] for batch in test_dl]
pred_sentences = sum(preds,[])

In [44]:
calculate_metric(
    src_sentences=src_sentences,
    tgt_sentences=tgt_sentences,
    pred_sentences=pred_sentences
)

({'S_D_p': 61.723,
  'S_D_r': 61.723,
  'S_D_f1': 61.723,
  'S_C_p': 32.238,
  'S_C_r': 32.238,
  'S_C_f1': 32.238,
  'C_D_p': 99.443,
  'C_D_r': 99.184,
  'C_D_f1': 99.313,
  'C_C_p': 95.237,
  'C_C_r': 94.99,
  'C_C_f1': 95.113},
 [61.723,
  61.723,
  61.723,
  32.238,
  32.238,
  32.238,
  99.443,
  99.184,
  99.313,
  95.237,
  94.99,
  95.113])